In [1]:
from itertools import product
from sage.rings.polynomial.pbori import declare_ring, Block

In [2]:
def rotate(reg, rot_n):
    rot_n = rot_n % len(reg)
    return tuple(reg[rot_n:] + reg[:rot_n])

def xor(l_reg, r_reg):
    assert len(l_reg) == len(r_reg)
    return tuple([ l+r for l,r in zip(l_reg, r_reg) ]) 

def add(l_reg, r_reg):
    assert len(l_reg) == len(r_reg)
    c = 0
    result = list()
    for l, r in zip(l_reg, r_reg):
        s = c + l + r
        c = (l*r) + (c*(l+r))
        result.append(s)
    return tuple(result)

def foo(left, right, side, left_rotation = 1, right_rotation = 1):
    assert len(left) == len(right)
    assert len(right) == len(side)
    
    left = rotate(left, left_rotation)
    left = add(left, right)
    left = xor(left, side)
    right = rotate(right, right_rotation)
    right = xor(left, right)
    return (left, right)

F = GF(2)
O, I = F(0), F(1)

def encrypt(K1, K2, PT1, PT2, rounds, left_rotation = 1, right_rotation = 1):
    word_size = len(K1)
    assert len(K2) == word_size
    assert len(PT1) == word_size
    assert len(PT2) == word_size

    nonces = list( product( (O, I), repeat = word_size ) ) #This generates consecutive integers in binary representation (nonces)
    assert rounds < len(nonces)
    
    for i, nonce in zip( range(rounds - 1), nonces):
        PT1, PT2 = foo(PT1, PT2, K2, left_rotation, right_rotation)
        K1, K2 = foo(K1, K2, nonce, left_rotation, right_rotation)
    
    PT1, PT2 = foo(PT1, PT2, K2, left_rotation, right_rotation)
    
    return (PT1, PT2)

def save_ES(ES, file_name):
    with open(file_name, "w") as file:
        for e in ES:
            for m in e:
                m = list(m)
                for v in m[:-1]:
                    file.write(f"{v.index()} ")
                if m:
                    file.write(f"{m[-1].index()}, ")
                else:
                    file.write(f", ")
            file.write("\n")

In [3]:
R = declare_ring( [Block('k', 4), Block('p', 4)] )

K = R.gens()[0:4]
P = R.gens()[4:8]
K1 = K[0:2]
K2 = K[2:4]
PT1 = P[0:2]
PT2 = P[2:4]

rounds = 2
CP1, CP2 = encrypt(K1, K2, PT1, PT2, rounds)
CP = (*CP1, *CP2)

K1 = (I, O) 
K2 = (I, O)
PT1 = (I, O)
PT2 = (O, I)
rounds = 2
CT1, CT2 = encrypt(K1, K2, PT1, PT2, rounds)
CT = (*CT1, *CT2)

K1 = K[0:2]
K2 = K[2:4]
PT1 = (I, O)
PT2 = (O, I)
ES = [ cp(*K1, *K2, *PT1, *PT2) + ct for cp, ct in zip(CP, CT) ]

save_ES(ES, "polynomials.txt")